In [22]:
import cvzone
from cvzone.HandTrackingModule import HandDetector
import cv2
import numpy as np
import pyautogui as control
import time
import mediapipe 
from cvzone.FPS import FPS
control.FAILSAFE =False
####### project Variable########### 
Wsc ,Hsc = control.size()
wCam, hCam = 640, 480
frameReduction = 100
plocationX ,plocationY = 0,0
clocationX ,clocationY = 0,0
smothingValue = 10



#define  detector
detector = HandDetector(staticMode=False,maxHands=2,modelComplexity=1,detectionCon=0.5, minTrackCon=0.5)                
##############variables#########

reader = FPS(avgCount=30)
cap = cv2.VideoCapture(0)
while True :
    _,frame = cap.read()
    hands,img = detector.findHands(frame,draw=False,flipType=True)
   #check hand is detected 
    if hands :
       rightHand =hands[0]
       if rightHand['type'] == 'Right'   :
           rightHandList = rightHand['lmList']
           # get needed fingure
           x1 ,y1 =rightHandList[8][0:2]
           x2 ,y2 =rightHandList[12][0:2]
           #get which fingure is up
           rightFingers = detector.fingersUp(rightHand)
          #creaet  screen reduction to can control mouse that reflect all screen and corners in main screen
           cv2.rectangle(frame,(frameReduction,frameReduction),(wCam-frameReduction,hCam-frameReduction),(255,0,255),2)
           #man
           if rightFingers [0]==0 and rightFingers[1]==1 :
               #start interpoliate pointer to main screen not in camera screen
               #x3 = np.interp(x1,(0,wCam),(0,Wsc))
              # y3 = np.interp(y1,(0,hCam),(0,Hsc))
               x3 = np.interp(x1,(frameReduction,wCam-frameReduction),(0,Wsc))
               y3 = np.interp(y1,(frameReduction,hCam-frameReduction),(0,Hsc))
               #start smothing process
               clocationX =plocationX +(x3-plocationX)/smothingValue
               clocationY =plocationY +(y3-clocationY)/smothingValue
               cv2.circle(frame,(x1,y1),5,(0,255,0),cv2.FILLED)
               control.moveTo(Wsc-clocationX,clocationY)
               plocationY = clocationY
               plocationX = clocationX

           if rightFingers[1]==1 and rightFingers[2]==1 :
             length, info, img = detector.findDistance(rightHandList[8][0:2],rightHandList[12][0:2] , frame, color=(255, 0, 255),scale=5) 
             if length<40 :
                 cv2.circle(frame,(info[4],info[5]),8,(0,255,255),cv2.FILLED)
                 control.click()
               #start right click  
           if rightFingers[3]==1 and rightFingers [4]==1  :
                control.rightClick()
           #start close camera 
           if rightFingers[0]==0 and rightFingers[1] ==0 and rightFingers[2]==0 and rightFingers[3]    and rightFingers[4]==0 : 
            cap.release()
            break                  
           
        
    
    imgPNG = cv2.imread("C:/Users/Geka/Desktop/update.png",cv2.IMREAD_UNCHANGED)
    imgOverlay = cvzone.overlayPNG(frame, imgPNG, pos=[3, 430])
    fps ,img = reader.update(frame,pos=(20,50),scale=1, thickness=2)
    cv2.imshow('ss',frame)
    key = cv2.waitKey(10)
    if key == ord('q'):
        break
cap.release()
cv2.destroyAllWindows() 
